In [1]:
import numpy as np

## Globals
α = 2.79 #1/m
θs = 0.385
θr = 0.012
n = 7.26
m = 1 - 1/n
Ks = 2.07E-4  #m/s
qtarget = 3.32E-6 #m/s

def vanGenuchten(h:np.array) -> np.array:
    θe = np.where(
        h >= 0,
        1.0,
        np.power(1 + np.power(α*h*np.sign(h), n), -m)
    )
    return θe

def waterSaturation(h:np.array) -> np.array:
    θe = vanGenuchten(h)
    θ = θe * (θs - θr) + θr
    return θ

def mualemPermeability(h:np.array) -> np.array:
    θe = vanGenuchten(h)
    kr = np.sqrt(θe) * np.power(1-np.power(1-np.power(θe, 1/m), m), 2)
    return kr

def capillarity(h) -> np.array:
    x = np.where(
        h >= 0,
        0.0,
        α*m*n * np.power(α*h*np.sign(h), n-1) * np.power(1 + np.power(α*h*np.sign(h), n), -m-1)
        )
    x *= (θs - θr)
    return x

In [2]:
def my_bisection(f, a, b, tol): 
    # approximates a root, R, of f bounded 
    # by a and b to within tolerance 
    # | f(m) | < tol with m the midpoint 
    # between a and b Recursive implementation
    
    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")
        
    # get midpoint
    m = (a + b)/2
    
    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on a. 
        # Make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on b. 
        # Make recursive call with b = m
        return my_bisection(f, a, m, tol)

In [4]:
kn = 1.0
def hydraulic_cond(h):
    return qtarget - (Ks * mualemPermeability(h) * kn)

my_bisection(hydraulic_cond, -10, 0, 1e-12)

-0.43241679668426514

In [ ]:
top
    {
        /*
        Rosenzweig (2011) Thesis
        ------------
        Flow rate               Q   = 1 mL/min = 1.67E-8 m³/s
        Hyd. conduct.           Ks  = 1.24 cm/min = 2.067E-4 m/s
                                K(h =-0.30) = 9.83E-5 m/s
        Diameter                D   = 8.0 cm = 0.08 m
        Cross-area              A   = π*D²/4
                                    = π*(0.08)²/4 = 5.026E-3 m²
        Darcy vel.               q  = Q/A
                                    = 1.67E-8 m³/s / 5.026E-3 m² = 3.32E-6 m/s

        -------
        Darcy Law - if no clogging and saturated
            q = - Ks • grad(h)
            3.32E-6 m/s = 2.067E-4 m/s • grad(h);
            grad(h) = 0.04345  [Check if this value was calculated by the code below!]

        */

        //- ♣ Fixed Darcy Flux

        type            codedMixed;
        refValue        uniform 0;
        refGradient     uniform 0;
        valueFraction   uniform 0;          // If 0: fixedGradient, if 1: fixedValue
        name            DarcyInflux;        // name of generated BC

        code
        #{
            // Read the hydraulic conductivity field
            const volScalarField& myK = this->db().objectRegistry::lookupObject<volScalarField>("hydraulicCond");

            // Specify the target Darcy flow velocity
            dimensionedScalar targetDarcyFlow 
            (
                "targetDarcyFlow",
                dimensionSet(0,1,-1,0,0,0,0),
                scalar(-3.32E-6) //Negative means inwards
            );

            int my_i = 0;
            // Assign the corresponding gradient to each boundary face
            forAll(patch(), faceI)
            { 
                Info << "Hydraulic conductivity at the patch is " << myK[faceI] << endl;
                Info << "Target Darcy flow is " << targetDarcyFlow.value() << endl;
                Info << my_i <<endl;
                my_i++;
                this->refGrad()[faceI] = targetDarcyFlow.value() / myK[faceI];

                Info << "grad(h) " << this->refGrad()[faceI] << endl;

            }

            //this->refGrad() = Zero;
            //this->valueFraction() = 1.0;
        #};
    }